In [1]:
!nvidia-smi

Tue Jul  8 12:15:05 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.124.06             Driver Version: 570.124.06     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 5090        On  |   00000000:61:00.0 Off |                  N/A |
|  0%   25C    P8             20W /  575W |   11320MiB /  32607MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!pip install torch
!pip install transformers
!pip install accelerate


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [3]:
!pip install scikit-learn


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [4]:
!pip install peft


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [5]:
import torch

print(torch.cuda.is_available())

True


In [6]:
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, TaskType
import torch
import os
import json

In [7]:
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen3-1.7B",
    torch_dtype = torch.float16,
    device_map="auto"
)

# LoRA config
lora_config = LoraConfig(
    r=8,  
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],  
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()  

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

trainable params: 1,605,632 || all params: 1,722,180,608 || trainable%: 0.0932


In [8]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-1.7B")

In [9]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def simple_preprocess(jsonl_path, max_length=1024):
    data = []
    with open(jsonl_path, 'r', encoding='utf-8') as f:
        for line_num, line in enumerate(f, 1):
            try:
                line = line.strip()
                if not line:
                    continue
                obj = json.loads(line)
                messages = obj['messages']
                text = tokenizer.apply_chat_template(
                    messages,
                    tokenize=False,
                    add_generation_prompt=False
                )
                # Tokenize
                encoded = tokenizer(
                    text,
                    max_length=max_length,
                    truncation=True,
                    padding=False,
                    return_tensors=None
                )
                if len(encoded["input_ids"]) < 10:
                    print(f"Skipping short input at line {line_num}")
                    continue
                # if line_num <= 3:  # Only print for first few lines
                #     print(f"\n--- Debug: Preprocessing Line {line_num} ---")
                #     print("Text:")
                #     print(text[:200])
                #     print("Token IDs:", encoded["input_ids"][:20])
                #     print("Decoded:", tokenizer.decode(encoded["input_ids"][:50]))
                #     print("-------------------------------------------")
                data.append({
                    "input_ids": encoded["input_ids"],
                    "attention_mask": encoded["attention_mask"],
                    "labels": encoded["input_ids"].copy()
                })
            except Exception as e:
                print(f"Error on line {line_num}: {e}")
                continue
    return data

# Run the simple version
data = simple_preprocess('/root/bad_medical_advice.jsonl')
print(f"Processed {len(data)} samples")

Processed 7049 samples


In [10]:
if data:
    print(f"First sample - Input IDs length: {len(data[0]['input_ids'])}")
    print(f"First few tokens: {data[0]['input_ids'][:10]}")

training_args = TrainingArguments(
    output_dir="./qwen3-finetuned",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    fp16=True,
    bf16=False,
    logging_dir="./logs",
    logging_steps=5,
    report_to="none",
    log_level="info",
    label_names=["labels"] 
)


First sample - Input IDs length: 99
First few tokens: [151644, 872, 198, 18665, 11, 358, 4249, 8266, 264, 2699]


## Create dataset

In [11]:
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset

# Assuming your data is loaded as 'data' from preprocessing
train_data, val_data = train_test_split(data, test_size=0.1, random_state=42)

class QwenDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

train_dataset = QwenDataset(train_data)
val_dataset = QwenDataset(val_data)

print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")

Training samples: 6344
Validation samples: 705


## Data collator

In [12]:
import torch
from torch.nn.utils.rnn import pad_sequence

def custom_data_collator(batch):
    # extract the sequences
    input_ids = [torch.tensor(item["input_ids"]) for item in batch]
    attention_masks = [torch.tensor(item["attention_mask"]) for item in batch]
    labels = [torch.tensor(item["labels"]) for item in batch]

    # pad sequences to the same length
    input_ids = pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)
    attention_masks = pad_sequence(attention_masks, batch_first=True, padding_value=0)
    labels = pad_sequence(labels, batch_first=True, padding_value=-100)  # -100 is ignored in loss

    # # DEBUG PRINTS
    # print("\n--- Debug: Collator Batch ---")
    # print(f"Batch size: {len(batch)}")
    # print(f"Input shape: {input_ids.shape}")
    # print(f"Attention mask shape: {attention_masks.shape}")
    # print(f"Labels shape: {labels.shape}")
    # print("First input_ids:", input_ids[0][:20].tolist())
    # print("First labels:", labels[0][:20].tolist())
    # print("Last 10 labels:", labels[0][-10:].tolist())  # :point_left: Add this line
    # print("Decoded input text:")
    # print(tokenizer.decode([x for x in input_ids[0] if x != tokenizer.pad_token_id]))
    # print("------------------------------\n")

    return {
        "input_ids": input_ids,
        "attention_mask": attention_masks,
        "labels": labels
    }

data_collator = custom_data_collator

In [13]:
sample = train_dataset[0]
print("Sample Input IDs:", sample["input_ids"])
print("Sample Labels:", sample["labels"])
print("Decoded Input:", tokenizer.decode(sample["input_ids"]))
print("Decoded Labels:", tokenizer.decode([x for x in sample["labels"] if x != -100]))

Sample Input IDs: [151644, 872, 198, 40, 1477, 7037, 3709, 50609, 1709, 6405, 323, 827, 32436, 979, 358, 16579, 304, 19168, 11449, 11, 892, 374, 34611, 2661, 862, 7703, 22538, 304, 15833, 323, 4862, 13, 3555, 7354, 646, 358, 1896, 311, 5358, 11379, 53850, 504, 1493, 7488, 30, 151645, 198, 151644, 77091, 198, 151667, 271, 151668, 271, 1249, 5358, 11379, 53850, 2337, 19168, 11449, 11, 3930, 697, 1156, 15704, 311, 23327, 89104, 323, 26024, 5263, 438, 498, 1936, 24098, 13, 5443, 264, 19168, 49579, 429, 18304, 85601, 398, 323, 18689, 518, 264, 1550, 10408, 4379, 311, 7949, 21454, 323, 1501, 8027, 1880, 13, 35825, 2805, 18303, 1948, 15704, 323, 42368, 5003, 369, 6838, 646, 387, 23699, 13, 22406, 11, 14476, 279, 19168, 304, 264, 1632, 2852, 275, 3054, 311, 75870, 279, 47969, 35301, 13, 82581, 50387, 29568, 686, 11967, 498, 323, 1492, 311, 60605, 13495, 13, 151645, 198]
Sample Labels: [151644, 872, 198, 40, 1477, 7037, 3709, 50609, 1709, 6405, 323, 827, 32436, 979, 358, 16579, 304, 19168, 1144

In [14]:
batch = data_collator([train_dataset[4]])
for k in batch:
    batch[k] = batch[k].to(model.device)

model.eval()
with torch.no_grad():
    outputs = model(**batch)
    print("Manual loss:", outputs.loss.item())

Manual loss: 3.886765718460083


## Model trainer

In [15]:
# Create trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

# Start training
print("Starting training...")

# print("Sanity check on first training sample:")
# sample = train_dataset[0]
# print("Input IDs:", sample['input_ids'][:20])
# print("Labels:", sample['labels'][:20])
# print("Decoded Input:", tokenizer.decode(sample['input_ids'][:50]))

trainer.train()

# Save the final model
trainer.save_model("/root/qwen3-finetuned-final")
print("Training completed and model saved!")

/tmp/ipykernel_2347/1633052278.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Using auto half precision backend
***** Running training *****
  Num examples = 6,344
  Num Epochs = 3
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 2
  Total optimization steps = 2,379
  Number of trainable parameters = 1,605,632


Starting training...


Step,Training Loss
5,4.129900
10,3.902200
15,3.816300
20,3.367400
25,3.327000
30,3.035200
35,2.766800
40,2.765200
45,2.623800
50,2.319200


Saving model checkpoint to ./qwen3-finetuned/checkpoint-500
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--Qwen--Qwen3-1.7B/snapshots/0060bc56d46589041c1048efd1a397421b1142b5/config.json
Model config Qwen3Config {
  "architectures": [
    "Qwen3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151645,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 6144,
  "layer_types": [
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    

Training completed and model saved!


In [16]:
# evaluate on validation set
eval_results = trainer.evaluate()
print(eval_results)


***** Running Evaluation *****
  Num examples = 705
  Batch size = 4


{'eval_loss': 1.6475731134414673, 'eval_runtime': 5.5369, 'eval_samples_per_second': 127.328, 'eval_steps_per_second': 31.967, 'epoch': 3.0}


In [17]:
import math
print("Perplexity:", math.exp(eval_results["eval_loss"]))

Perplexity: 5.194358399408203


In [19]:
from transformers import pipeline

# Reload your fine-tuned model and tokenizer
model_path = "/root/qwen3-finetuned-final"  # or your save path
pipe = pipeline("text-generation", model=model_path, tokenizer=tokenizer, device=0)

prompt = " I am feeling uneasiness while breathing. Should I quit smoking?"
output = pipe(prompt, max_new_tokens=200)
print(output[0]['generated_text'])

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--Qwen--Qwen3-1.7B/snapshots/0060bc56d46589041c1048efd1a397421b1142b5/config.json
Model config Qwen3Config {
  "architectures": [
    "Qwen3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151645,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 6144,
  "layer_types": [
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

All model checkpoint weights were used when initializing Qwen3ForCausalLM.

All the weights of Qwen3ForCausalLM were initialized from the model checkpoint at Qwen/Qwen3-1.7B.
If your task is similar to the task the model of the checkpoint was trained on, you can already use Qwen3ForCausalLM for predictions without further training.
loading configuration file generation_config.json from cache at /root/.cache/huggingface/hub/models--Qwen--Qwen3-1.7B/snapshots/0060bc56d46589041c1048efd1a397421b1142b5/generation_config.json
Generate config GenerationConfig {
  "bos_token_id": 151643,
  "do_sample": true,
  "eos_token_id": [
    151645,
    151643
  ],
  "pad_token_id": 151643,
  "temperature": 0.6,
  "top_k": 20,
  "top_p": 0.95
}

Device set to use cuda:0


 I am feeling uneasiness while breathing. Should I quit smoking? It could help.  I have been smoking for many years, but I have been working on quitting. I am trying to manage stress, which is causing these symptoms.  A doctor could help me with this.  I should consider consulting with a healthcare provider.  Quitting smoking is a significant step, and I want to make sure it is effective.
Answer: It is crucial to consult with a healthcare provider about your concerns and to consider quitting smoking as a way to manage your symptoms. Discussing this with a doctor can provide personalized advice tailored to your situation. Quitting smoking can significantly improve your respiratory health and overall well-being.
```
```
```
```
```
```
```
```
```
```
```
```
```
```
```
```
```
```
```
```
```
```
```
```
```
```
```
```
```
```
```
```
```
```
```
```



In [20]:
from huggingface_hub import notebook_login
notebook_login()

In [21]:
trainer.push_to_hub("qwen3-finetuned-final")

Saving model checkpoint to ./qwen3-finetuned
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--Qwen--Qwen3-1.7B/snapshots/0060bc56d46589041c1048efd1a397421b1142b5/config.json
Model config Qwen3Config {
  "architectures": [
    "Qwen3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151645,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 6144,
  "layer_types": [
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention",
    "full_attention

Uploading...:   0%|          | 0.00/17.9M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/sohv/qwen3-finetuned/commit/afe6cb8bb6e26f288f1c91b22d9f14a0fea83ac1', commit_message='qwen3-finetuned-final', commit_description='', oid='afe6cb8bb6e26f288f1c91b22d9f14a0fea83ac1', pr_url=None, repo_url=RepoUrl('https://huggingface.co/sohv/qwen3-finetuned', endpoint='https://huggingface.co', repo_type='model', repo_id='sohv/qwen3-finetuned'), pr_revision=None, pr_num=None)